In [1]:
import pandas as pd
import numpy as np

lectures_dtypes = {
    # foreign key for the train.content_id column, when the content type is lecture (1)
    "lecture_id": "uint64",
    # one tag codes for the lecture. The meaning of the tags will not be provided, but these codes are sufficient for clustering the lectures together
    "tag": "int16",
     # top level category code for the lecture
    "part": "uint8",
    # brief description of the core purpose of the lecture
    "type_of": "object",
}

questions_dtypes = {
    # foreign key for the train.content_id column, when the content type is question (0)
    "question_id": "uint64",
    # code for which questions are served together
    "bundle_id": "uint32",
    # the answer to the question. Can be compared with the train user_answer column to check if the user was right
    "correct_answer": "uint8",
    # the relevant section of the TOEIC test
    "part": "uint8",
    # one or more detailed tag codes for the question. The meaning of the tags will not be provided, but these codes are sufficient for clustering the questions together
    "tags": "object",
}                                                                                                  

train_dtypes = {
    # ID code for the row
    "row_id": "int64",
    # the time in milliseconds between this user interaction and the first event completion from that user
    "timestamp": "int64",
    # ID code for the user
    "user_id": "uint32",
    # ID code for the user interaction
    "content_id": "int16",
    # 0 if the event was a question being posed to the user, 1 if the event was the user watching a lecture
    "content_type_id": "boolean",
    # Id code for the batch of questions or lectures. For example, a user might see three questions in a row before seeing the explanations for any of them. Those three would all share a task_container_id
    "task_container_id": "int16",
    # the user's answer to the question, if any. Read -1 as null, for lectures
    "user_answer": "int8",
    # if the user responded correctly. Read -1 as null, for lectures
    "answered_correctly": "int8",
    # The average time in milliseconds it took a user to answer each question in the previous question bundle, ignoring any lectures in between. Is null for a user's first question bundle or lecture. Note that the time is the average time a user took to solve each question in the previous bundle
    "prior_question_elapsed_time": "float32",
    # Whether or not the user saw an explanation and the correct response(s) after answering the previous question bundle, ignoring any lectures in between. The value is shared across a single question bundle, and is null for a user's first question bundle or lecture. Typically the first several questions a user sees were part of an onboarding diagnostic test where they did not get any feedback
    "prior_question_had_explanation": "boolean"
}

lectures_original_df = pd.read_csv('data/lectures.csv', dtype=lectures_dtypes)
questions_original_df = pd.read_csv('data/questions.csv', dtype=questions_dtypes)
train_original_df = pd.read_csv('data/train.csv', dtype=train_dtypes)

# Знакомство с данными

## Лекции

In [4]:
display(lectures_original_df.info())
display(lectures_original_df.head())
display(lectures_original_df.tail())
display(lectures_original_df.describe())
display(lectures_original_df.describe(include=['object']))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   lecture_id  418 non-null    uint64
 1   tag         418 non-null    int16 
 2   part        418 non-null    uint8 
 3   type_of     418 non-null    object
dtypes: int16(1), object(1), uint64(1), uint8(1)
memory usage: 7.9+ KB


None

,lecture_id,tag,part,type_of
0,89,159,5,concept
1,100,70,1,concept
2,185,45,6,concept
3,192,79,5,solving question
4,317,156,5,solving question


,lecture_id,tag,part,type_of
413,32535,8,5,solving question
414,32570,113,3,solving question
415,32604,24,6,concept
416,32625,142,2,concept
417,32736,82,3,concept


,lecture_id,tag,part
count,418.000000,418.000000,418.000000
mean,16983.401914,94.480861,4.267943
std,9426.164660,53.586487,1.872424
min,89.000000,0.000000,1.000000
25%,9026.250000,50.250000,2.000000
50%,17161.500000,94.500000,5.000000
75%,24906.250000,140.000000,6.000000
max,32736.000000,187.000000,7.000000


,type_of
count,418
unique,4
top,concept
freq,222


In [5]:
# все столбы заполнены, убедимся в этом
print(lectures_original_df.isnull().sum().to_frame().T)

   lecture_id  tag  part  type_of
0           0    0     0        0


In [6]:
# посмотрим какие типы лекций бывают
display(lectures_original_df['type_of'].value_counts().to_frame().T)

type_of,concept,solving question,intention,starter
count,222,186,7,3


## Вопросы

In [8]:
display(questions_original_df.info())
display(questions_original_df.head())
display(questions_original_df.tail())
display(questions_original_df.describe())
display(questions_original_df.describe(include=['object']))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13523 entries, 0 to 13522
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   question_id     13523 non-null  uint64
 1   bundle_id       13523 non-null  uint32
 2   correct_answer  13523 non-null  uint8 
 3   part            13523 non-null  uint8 
 4   tags            13522 non-null  object
dtypes: object(1), uint32(1), uint64(1), uint8(2)
memory usage: 290.7+ KB


None

,question_id,bundle_id,correct_answer,part,tags
0,0,0,0,1,51 131 162 38
1,1,1,1,1,131 36 81
2,2,2,0,1,131 101 162 92
3,3,3,0,1,131 149 162 29
4,4,4,3,1,131 5 162 38


,question_id,bundle_id,correct_answer,part,tags
13518,13518,13518,3,5,14
13519,13519,13519,3,5,8
13520,13520,13520,2,5,73
13521,13521,13521,0,5,125
13522,13522,13522,3,5,55


,question_id,bundle_id,correct_answer,part
count,13523.00000,13523.000000,13523.000000,13523.000000
mean,6761.00000,6760.510907,1.455298,4.264956
std,3903.89818,3903.857783,1.149707,1.652553
min,0.00000,0.000000,0.000000,1.000000
25%,3380.50000,3379.500000,0.000000,3.000000
50%,6761.00000,6761.000000,1.000000,5.000000
75%,10141.50000,10140.000000,3.000000,5.000000
max,13522.00000,13522.000000,3.000000,7.000000


,tags
count,13522
unique,1519
top,8
freq,738


## Действия 

In [ ]:
display(train_original_df.info())
display(train_original_df.head())
display(train_original_df.tail())
display(train_original_df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101230332 entries, 0 to 101230331
Data columns (total 10 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   row_id                          int64  
 1   timestamp                       int64  
 2   user_id                         uint32 
 3   content_id                      int16  
 4   content_type_id                 boolean
 5   task_container_id               int16  
 6   user_answer                     int8   
 7   answered_correctly              int8   
 8   prior_question_elapsed_time     float32
 9   prior_question_had_explanation  boolean
dtypes: boolean(2), float32(1), int16(2), int64(2), int8(2), uint32(1)
memory usage: 3.2 GB


None

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,0,115,5692,False,1,3,1,NaN,<NA>
1,1,56943,115,5716,False,2,2,1,37000.0,False
2,2,118363,115,128,False,0,0,1,55000.0,False
3,3,131167,115,7860,False,3,0,1,19000.0,False
4,4,137965,115,7922,False,4,1,1,11000.0,False


,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
101230327,101230327,428564420,2147482888,3586,False,22,0,1,18000.0,True
101230328,101230328,428585000,2147482888,6341,False,23,3,1,14000.0,True
101230329,101230329,428613475,2147482888,4212,False,24,3,1,14000.0,True
101230330,101230330,428649406,2147482888,6343,False,25,1,0,22000.0,True
101230331,101230331,428692118,2147482888,7995,False,26,3,1,29000.0,True


In [ ]:
# Пропуски в данных
print(train_original_df.isnull().sum())

# Анализ данных

In [ ]:
lectures_df = lectures_original_df.copy() 
questions_df = questions_original_df.copy() 
train_df = train_original_df.copy() 

train_questions_df = train_df[train_df['content_type_id'] == False]
train_lectures_df = train_df[train_df['content_type_id'] == True]

questions_total_count = train_questions_df.shape[0]
questions_correct_count = train_questions_df[train_questions_df['answered_correctly'] == 1].shape[0]
lectures_count = train_lectures_df.shape[0]

print(f"Сколько пройденных заданий({questions_total_count}) приходится на прослушанные лекции({lectures_count}): {round(questions_count / lectures_count)}")
print(f"Процент правильных ответов: {questions_correct_count / questions_count * 100:.0f}%")

Попробуем выделить отчего зависит успеваемость студента

In [ ]:
# Влияние объяснений на правильность ответа(есть)
train_df[train_df['prior_question_had_explanation'].notnull()].groupby('prior_question_had_explanation')['answered_correctly'].mean()

In [ ]:
# Влияние скорости ответа на правильность(нет)
grouped_by_time_df = train_df[train_df['prior_question_elapsed_time'].notnull()].groupby('answered_correctly')['prior_question_elapsed_time']
display(grouped_by_time_df.mean())
display(grouped_by_time_df.quantile(np.linspace(0.5, 0.95, 5)))

Попробуем проанализировать сложность вопросов. Самые простые(все ответили) и самые сложные(никто не ответил) имеет смысл пересмотреть.

In [ ]:
# Объединение данных для анализа вопросов
merged_questions_df = pd.merge(train_df[train_df['content_type_id'] == 0], questions_df, left_on='content_id', right_on='question_id')
merged_questions_df = merged_questions_df.drop(columns=['content_id', 'content_type_id'])

questions_difficulty = merged_questions_df.groupby('question_id').agg(
    total_answers=('answered_correctly', 'count'),
    correct_answers=('answered_correctly', lambda x: (x == 1).sum())
).reset_index()
questions_difficulty['accuracy'] = (questions_difficulty['correct_answers'] / questions_difficulty['total_answers'] * 100).round()

print(f'{questions_difficulty['accuracy'].mean():.2f}')

# слишком сложные вопросы
questions_difficulty[questions_difficulty['accuracy'] > 99].sort_values(by=['accuracy', 'total_answers'], ascending=False).head(10)